# Proganomly Inference Pipeline

## Packages

TensorFlow and numpy should already be installed.

In [ ]:
%%bash
sudo apt-get update
sudo apt-get --assume-yes install openslide-tools
sudo apt-get --assume-yes install python-openslide
pip3 install --upgrade pip
pip3 install opencv-python-headless
pip3 install openslide-python
pip3 install matplotlib
pip3 install scikit-image
pip3 install shapely

## Configs

There are six main groups of configs: input, output, GAN inference, segmentation inference, polygon inference, and Dataflow.

The input configs are where input files are located. As requested there are several possible paths.

1. If `pre_computed_image_gcs_path` is not empty, inference from the model will be skipped and images at that GCS path will be read and used to create polygons.
1. If `png_individual_gcs_glob_pattern` is not empty, query image patch files will be read from GCS using this glob pattern and will call the model for inference and will create polygons at the end. To increase speed of inference, we can copy the SavedModel to a local directory and set `export_dir` to that path. Don't forget to then change `exports_on_gcs` to `True`.
1. If `png_patch_stitch_gcs_path` or `wsi_stitch_gcs_path` is not empty then the Dataflow stitching pipeline will be called where all of the patches will call the model for inference and then be stitched together. Since this can take quite some time, the notebook may lose connection, despite the Dataflow job still running perfectly in the background. If this happens, then change `pre_computed_image_gcs_path` to be what `gcs_output_image_filepattern` was previously to use the results of the Dataflow pipeline. `png_patch_stitch_gcs_path` and `wsi_stitch_gcs_path` cannot both be set at the same time.
  1. If `png_patch_stitch_gcs_path` is not empty, then the Dataflow stitching pipeline will use pre-extracted patch PNG files at the GCS path for inference and then stitch after.
  1. If `wsi_stitch_gcs_path` is not empty, then the Dataflow stitching pipeline will use the WSI at that path to extract patches in memory and then stitch after inference.

The output config contains boolean flags for which output types are desired as well as the GCS output path `gcs_output_image_filepattern`.

The GAN inference config contains all hyperparamters needed for GAN inference.

The segmentation inference config contains all hyperparamters needed for segmentation inference.

The polygon inference config contains all hyperparamters needed for polygon inference.

The dataflow config contains the parameters needed for the Dataflow sitching pipeline, only used if Dataflow gets called.

In [ ]:
input_config = {
    "slide_name": "slide_name",
    "use_pre_computed_gcs_paths": True,
    # berg only.
#     "pre_computed_generated_images_gcs_path": "",
#     "pre_computed_encoded_generated_images_gcs_path": "",
#     "pre_computed_query_encoded_images_gcs_path": "",
    # GANomaly only.
#     "pre_computed_query_gen_encoded_images_gcs_path": "",
    # Both.
#     "pre_computed_query_images_gcs_path": "",
#     "pre_computed_query_anomaly_images_linear_rgb_gcs_path": "",
#     "pre_computed_query_anomaly_images_linear_gs_gcs_path": "",
#     "pre_computed_query_mahalanobis_distance_images_linear_gcs_path": "",
#     "pre_computed_query_pixel_anomaly_flag_images_gcs_path": "",
#     "pre_computed_kde_rgb_gcs_path": "",
#     "pre_computed_kde_gs_gcs_path": "",
#     "pre_computed_kde_gs_thresholded_gcs_path": "",
#     "pre_computed_segmentation_cell_coords_gcs_path": "",
#     "pre_computed_segmentation_nuclei_coords_gcs_path": "",
#     "pre_computed_patch_coordinates_gcs_path": "",

    "png_individual_gcs_glob_pattern": "",
    "png_patch_stitch_gcs_glob_pattern": "",
    "wsi_stitch_gcs_path": ""
}
output_config = {
    # berg only.
    "output_generated_images": False,
    "output_encoded_generated_images": False,
    "output_query_encoded_images": False,
    # GANomaly only.
    "output_query_gen_encoded_images": False,
    # Both.
    "output_query_images": False,
    "output_query_anomaly_images_linear_rgb": False,
    "output_query_anomaly_images_linear_gs": False,
    "output_query_mahalanobis_distance_images_linear": False,
    "output_query_pixel_anomaly_flag_images": False,
    "output_kde_rgb": False,
    "output_kde_gs": False,
    "output_kde_gs_thresholded": False,
    # Requires query_images, kde_gs, and patch_coordinates.
    "output_kde_gs_polygon": True,
    "output_segmentation_cell_coords": False,
    "output_segmentation_nuclei_coords": False,
    "output_patch_coordinates": False,
    # Used if running a beam stitch job. Don't forget the slash on the end.
    "output_gcs_path": "gs://.../test/"
}
gan_inference_config = {
    # Approximate width of resultant thumbnail image.
    "target_image_width": 500,
    # Method to use for converting thumbnail of slide into binary mask. Either
    # otsu or rgb2hed.
    "thumbnail_method": "otsu",
    # Threshold to convert RGB2HED image to binary mask.
    "rgb2hed_threshold": -0.41,
    # Threshold to compare with percent of binary flags within a patch region
    # to include in collection.
    "include_patch_threshold": 0.5,
    # Whether SavedModel exports are on GCS or local.
    "exports_on_gcs": True,
#     "exports_on_gcs": False,
    # GAN SavedModel export directory on GCS or local filesystem.
    "gan_export_dir": "gs://.../trained_models",
#     "gan_export_dir": "../../trained_models",
    # Folder name of exported GAN SavedModel.
    "gan_export_name": "dynamic_threshold",
    # Generator architecture type, 'berg' or 'GANomaly'.
    "generator_architecture": "GANomaly",
    # For berg architecture, whether to use Z inputs. Query image inputs are
    # always used.
    "berg_use_Z_inputs": True,
    # For berg architecture, the latent size of the noise vector.
    "berg_latent_size": 512,
    # For berg architecture, the latent vector's random normal mean.
    "berg_latent_mean": 0.0,
    # For berg architecture, the latent vector's random normal standard
    # deviation.
    "berg_latent_stddev": 1.0,
    # Bandwidth of the kernel.
    "bandwidth": 100.0,
    # Kernel to use for density estimation.
    "kernel": "gaussian",
    # Distance metric to use. Note that not all metrics are valid with all
    # algorithms.
    "metric": "euclidean",
    # Number of sample bins to create in the x dimension.
    "xbins": 100,
    # Number of sample bins to create in the y dimension.
    "ybins": 100,
    # Minimum number of adjacent points as not to be removed from image.
    "min_neighborhood_count": 50,
    # Connectivity defining the neighborhood of a pixel.
    "connectivity": 10,
    # Minimum number of anomaly points following removing small objects to not
    # clear all flags.
    "min_anomaly_points_remaining": 200,
    # Exponent to use for scaling.
    "scaling_power": 0.5,
    # Positive factor to scale anomaly flag counts by.
    "scaling_factor": 100000.0,
    # Which color map to use.
    "cmap_str": "turbo",
    # Amount to scale the bandwidth based on anomaly counts. If 0.0, no scaling.
    "dynamic_bandwidth_scale_factor": 50000.0,
    # Maximum number of points allowed to run KDE. Otherwise entire image is marked as anomalous.
    "max_anomaly_points_for_kde": 50000,
    # Threshold to convert KDE grayscale image into binary mask to create image.
    "kde_threshold": 0.2,
    # Threshold to override learned Mahalanobis distance threshold from
    # SavedModel for creating Mahalanobis binary mask.
    "custom_mahalanobis_distance_threshold": 2.8172882 + 3.0 * 67.66342,  # mu + num_sigma * stddev
    # Depth of n-ary tree for GAN image stitching.
    # Let's say you have a slide that is 86000 x 112000. This means, if my
    # patches are 1024 x1024, that 83.984 ~ 83 patches can fit in the x
    # dimension and 109.375 ~ 109 patches can fit in the y dimension. However,
    # I need to stitch cleanly a left and a right patch (power of 2 in the x
    # dimension) and an up and a down patch (power of 2 in the y dimension).
    # Therefore the next closest biggest power of 2 in the x dimension is
    # 83 -> 128 and in the y is 109 -> 128. This results in a 128 x 128 patch
    # image. Even though this is already square, in case it is not, we take
    # the max of each dimension and then set both to that.

    # log(128, 2) = 7. That is where the 7 comes from. The stitching will
    # require a depth of 7 of the 4-ary tree to complete the slide.
    # Depth: 7, Size 128x128
    # Depth: 6, Size 64x64
    # Depth: 5, Size 32x32
    # Depth: 4, Size 16x16
    # Depth: 3, Size 8x8
    # Depth: 2, Size 4x4
    # Depth: 1, Size 2x2
    # Depth: 0, Size 1x1
    "nary_tree_depth": 7  # most use 7, 1805.WT uses 8, 41.BRAF_V600E uses 6
    # List of 2-tuples of output image height and width for each n-ary level, starting from leaves.
    "output_image_sizes": [(1024, 1024)] * 10,
}
# Segmentation inference config only used if outputting:
# output_segmentation_cell_coords or output_segmentation_nuclei_coords.
segmentation_inference_config = {
    # Directory containing exported segmentation models.
    "segmentation_export_dir": "gs://.../nuclei_segmentation_model",
    # Name of segmentation model
    "segmentation_model_name": "segmentation_model_name.meta",
    # Size of each patch of image for segmentation model.
    "segmentation_patch_size": 128,
    # Number of pixels to skip for each patch of image for segmentation model.
    "segmentation_stride": 16,
    # Whether to median blur images before segmentation.
    "segmentation_median_blur_image": False,
    # Kernel size of median blur for segmentation.
    "segmentation_median_blur_kernel_size": 9,
    # Number of patches to include in a group for segmentation.
    "segmentation_group_size": 10
}
inference_config = {
    # Number of images to include in each batch for inference.
    "batch_size": 8,
    # Height in pixels of a patch.
    "patch_height": 1024,
    # Width in pixels of a patch.
    "patch_width": 1024,
    # Number of channels of an image patch.
    "patch_depth": 3,
    "gan": gan_inference_config,
    "segmentation": segmentation_inference_config
}
polygon_config = {
    # Threshold to convert KDE grayscale image into binary mask to create
    # Polygon.
    "kde_gs_polygon_threshold": 0.2,
    # Factor to scale/dilate the `MultiPolygon`.
    "kde_gs_polygon_dilation_factor": 1.0,
    # The origin each polygon should be scaled about. 'center' or 'centroid'.
    "dilation_origin": "centroid",
    # Whether to limit Polygon vertices to only lie within patches. Requires
    # patch coordinates list. For stitched slide images, set to True.
    # Otherwise, for individual PNG images, set to False.
    "limit_polygon_vertices_to_only_patches": True,
    # Let's say you have a slide that is 86000 x 112000. This means, if my
    # patches are 1024 x1024, that 83.984 ~ 83 patches can fit in the x
    # dimension and 109.375 ~ 109 patches can fit in the y dimension. However,
    # I need to stitch cleanly a left and a right patch (power of 2 in the x
    # dimension) and an up and a down patch (power of 2 in the y dimension).
    # Therefore the next closest biggest power of 2 in the x dimension is
    # 83 -> 128 and in the y is 109 -> 128. This results in a 128 x 128 patch
    # image. Even though this is already square, in case it is not, we take
    # the max of each dimension and then set both to that.

    # log(128, 2) = 7. That is where the 7 comes from. The stitching will
    # require a depth of 7 of the 4-ary tree to complete the slide.
    # Depth: 7, Size 128x128
    # Depth: 6, Size 64x64
    # Depth: 5, Size 32x32
    # Depth: 4, Size 16x16
    # Depth: 3, Size 8x8
    # Depth: 2, Size 4x4
    # Depth: 1, Size 2x2
    # Depth: 0, Size 1x1

    # I need to be able to reconstruct that size.
    # Therefore, (num_patches) * (patch_size) = (2 ** 7) * (1024)
    "effective_slide_height": 2 ** 7 * 1024,
    "effective_slide_width": 2 ** 7 * 1024
}
# Dataflow config only used if calling a Dataflow stitch job.
dataflow_config = {
    # Project to run the Dataflow job .
    "project": "...",
    # GCS bucket to stage temporary files.
    "bucket": "gs://...",
    # Region to run the Dataflow job, make sure you have quota.
    "region": "us-central1",
    # Autoscaling mode for Dataflow job. Possible values are THROUGHPUT_BASED
    # to enable autoscaling or NONE to disable.
    "autoscaling_algorithm": "NONE",
    # Initial number of Google Compute Engine instances to use when executing
    # your pipeline. This option determines how many workers the Dataflow
    # service starts up when your job begins.
    "num_workers": 60,
    # Compute Engine machine type that Dataflow uses when starting worker VMs.
    "machine_type": "n1-highmem-32",
    # Disk size, in gigabytes, to use on each remote Compute Engine worker instance.
    "disk_size_gb": 1000,
    # Specifies a user-managed controller service account, using the format
    # my-service-account-name@<project-id>.iam.gserviceaccount.com.
    "service_account_email": "...",
    # Specifies whether Dataflow workers use public IP addresses. If the value
    # is set to false, Dataflow workers use private IP addresses for all
    # communication. In this case, if the subnetwork option is specified, the
    # network option is ignored. Make sure that the specified network or
    # subnetwork has Private Google Access enabled. Public IP addresses have
    # an associated cost.
    "use_public_ips": False,
    # Compute Engine network for launching Compute Engine instances to run
    # your pipeline.
    "network": "https://...",
    # Compute Engine subnetwork for launching Compute Engine instances to run
    # your pipeline.
    "subnetwork": "https://...",
    # Runner of pipeline. "DirectRunner" for running local, "DataflowRunner"
    # for running distributed Dataflow job.
    "runner": "DataflowRunner"  # Directrunner or DataflowRunner
}
config = {
    "input": input_config,
    "output": output_config,
    "inference": inference_config,
    "polygon": polygon_config,
    "dataflow": dataflow_config
}

In [ ]:
config

## Inference Pipeline

This is the entrypoint code for the inference pipeline. As mentioned above there are many conditional paths depending on what was set in the config.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from proganomaly_modules.inference_module import inference

## Run Pipeline

We can now run the inference pipeline with our configs!

In [ ]:
output_dict = inference.inference_pipeline(config)

In [ ]:
# %%bash
# ./run_beam.sh

In [ ]:
output_dict.keys()

## Inspect Outputs

With our pipeline run, we can now inspect the output dictionary.

In [ ]:
from proganomaly_modules.inference_module import image_utils

In [ ]:
image_utils.plot_images(
    images=image_utils.descale_images(images=output_dict["query_images"]),
    depth=3,
    num_rows=1
)

In [ ]:
image_utils.plot_images(
    images=image_utils.descale_images(images=output_dict["kde_rgb"]),
    depth=3,
    num_rows=1
)

In [ ]:
image_utils.plot_images(
    images=image_utils.descale_images(images=output_dict["kde_gs"]),
    depth=1,
    num_rows=1
)

In [ ]:
for i, polygon in enumerate(output_dict["kde_gs_polygon"]):
    image_utils.plot_geometry_contours(
        geometry=polygon,
        image=output_dict["query_images"][i],
        fig_name=""
    )

## Plot Factory

In [ ]:
for thresh in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.5]:
    for dil in [4.0]:
        config["polygon"]["kde_gs_polygon_threshold"] = thresh
        config["polygon"]["kde_gs_polygon_dilation_factor"] = dil
        output_dict = inference.inference_pipeline(config)
        image_utils.plot_geometry_contours(
            geometry=output_dict["kde_gs_polygon"][0],
            image=output_dict["query_images"][0],
            fig_name="18.WT_MHD3.0_KDEt{thresh}_DIL{dil}_MINPIX500".format(
                thresh=thresh,dil=dil
            )
        )